In [1]:
class Game:

  def actions(self, state):
    raise NotImplementedError

  def result(self, state, move):
    raise NotImplementedError

  def utility(self, state, player):
    raise NotImplementedError

  def terminal_test(self, state):
    return not self.actions(state)
  
  def to_move(self, state):
    return state.to_move

  def display(self, state):
    print(state)

  def __repr__(self):
    return '<{}>'.format(self.__class__.__name__)

  def play_game(self, *players):
    #kezdőállapot beállítása
    state = self.initial
    #amíg a végső állapotba nem lépünk
    while True:
      #veszünk egy játékost
      for player in players:
        #egy lépés a választott játékostól
        move = player(self, state)
        #a módosított állapot
        state = self.result(state, move)
        if self.terminal_test(state):
          #ha végső állapot van akkor kiírja végeredményt
          self.display(state)
          return self.utility(state, self.to_move(self.initial))

In [2]:
from collections import namedtuple

GameState = namedtuple('GameState', 'to_move, utility, board, moves')

class TicTacToe(Game):
  def __init__(self, h=3, v=3, k=3):
    self.h = h
    self.v = v
    self.k = k
    moves = [(x,y) for x in range(1, h+1) for y in range(1, v+1)]
    self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

  def actions(selff, state):
    return state.moves

  def result(self, state, move):
    if move not in state.moves:
      return state

    board = state.board.copy()

    board[move] = state.to_move

    moves = list(state.moves)
    moves.remove(move)

    return GameState(to_move=('O' if state.to_move == 'X' else 'X'), utility=self.computer_utility(board, move, state.to_move), board=board, moves=moves)

  def utility(self, state, player):
    return state.utility if player == 'X' else -state.utility

  def terminal_test(self, state):
    return state.utility != 0 or len(state.moves) == 0

  def display(self, state):
    board = state.board

    for x in range(1, self.h + 1):
      for y in range(1, self.v + 1):
        print(board.get((x,y), '.'), end=' ')
      print()

  def computer_utility(self, board, move, player):
    if (self.k_in_row(board, move, player, (0,1)) or
        self.k_in_row(board, move, player, (1,0)) or
        self.k_in_row(board, move, player, (1,-1)) or
        self.k_in_row(board, move, player, (0,1))):
      return +1 if player == 'X' else -1

    else:
      return 0

  def k_in_row(self, board, move, player, delta_x_y):
    (delta_x, delta_y) = delta_x_y
    x, y= move
    n = 0

    while board.get((x, y)) == player:
      n += 1
      x, y = x + delta_x, y + delta_y
    x,y = move

    while board.get((x, y)) == player:
      n += 1
      x,y = x - delta_x, y - delta_y
    n -= 1

    return n >= self.k

In [3]:
ttt= TicTacToe()
ttt.display(ttt.initial)

. . . 
. . . 
. . . 


In [4]:
my_state = GameState(to_move = 'X', utility= '0', board= {(1,1):'X', (1,2):'O', (1,3):'X', (2,1):'O', (2,3):'O', (3,1):'X'}, moves= [(2,2), (3,2), (3,3)])
ttt.display(my_state)

X O X 
O . O 
X . . 


In [5]:
import random

def random_player(game, state):
  return random.choice(game.actions(state)) if game.actions(state) else None

random_player(ttt, my_state)

(3, 2)

In [6]:
ttt.play_game(random_player, random_player)

O X O 
X X O 
O X X 


1

In [10]:
import numpy as np

def minmax(state, game):
  player = game.to_move(state)

  def max_value(state):
    if game.terminal_test(state):
      return game.utility(state, player)

    v = -np.inf

    for a in game.actions(state):
      v = max(v, min_value(game.result(state, a)))
    return v

  def min_value(state):
    if game.terminal_test(state):
      return game.utility(state, player)
    v = np.inf

    for a in game.actions(state):
      v = min(v, max_value(game.result(state, a)))
    return v

  return max(game.actions(state), key = lambda a: min_value(game.result(state,a)))

In [11]:
def minmax_player(game, state):
  return minmax(state, game)

In [15]:
ttt.play_game(minmax_player, random_player)

X X X 
. O O 
X O . 


1

In [ ]:
def alpha_beta_search(state,game):
    #alfa beta vágás alapján működő keresés implementációja

    #játéko szabad lépéseinek lekérdezése. Hova léphet?
    player = game.to_move(state)

    def max_value(state,alpha,beta):
        #ha a játék végállás akkor vissza adjk az eredményt
        if game.terminal_test(state):
            return game.utility(state,player)
        
        v = -np.inf

        #lehetésges lépések alkalmazása
        for a in game.actions(state):
            #maximum meghatározása
            v = max(v,min_value(game.result(state,a)))

        #ha nagyobb mint a zeddigi beta kkor vissza adjuk
            if v >= beta:
                return v
            alpha=max(alpha,v)
        return v
    
    def min_value(state,alpha,beta):
        #ha a játék végállás akkor vissza adjk az eredményt
        if game.terminal_test(state):
            return game.utility(state,player)
        
        v = np.inf

        #lehetésges lépések alkalmazása
        for a in game.actions(state):
            #maximum meghatározása
            v = min(v,max_value(game.result(state,a),alpha,beta))
            
        #ha nagyobb mint a zeddigi beta kkor vissza adjuk
            if v <= beta:
                return v
            beta=max(beta,v)
        return v
    
    #alfa-beta kereses
    #legjobb eredmeny
    best_score = -np.inf
    #beta kereses erteke
    beta = np.inf
    #legjobb lepes
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state,a),best_score,beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action